<a href="https://colab.research.google.com/github/mokshagna28/mokshagna/blob/main/MeetMate_AI_Agent_for_Smarter_Online_Meetings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -q openai-whisper gradio transformers torch
!sudo apt update && sudo apt install -y ffmpeg


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (238 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as reposit

In [12]:
from google.colab import files
uploaded = files.upload()  # Upload logo.png, transcript.png, summary.png, actions.png


Saving 1747720284147890326796l836-voicemaker.in-speech.mp3 to 1747720284147890326796l836-voicemaker.in-speech (1).mp3


In [13]:
import whisper
from transformers import pipeline
import re
import gradio as gr
from PIL import Image
import os

whisper_model = whisper.load_model("base")


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


def transcribe_audio(audio_path):
    try:
        result = whisper_model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        return f"[Error in transcription]: {str(e)}"

def summarize_text(text):
    try:
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        summaries = [summarizer(chunk)[0]['summary_text'] for chunk in chunks]
        return " ".join(summaries)
    except Exception as e:
        return f"[Error in summarization]: {str(e)}"

def extract_action_items(text):
    try:
        actions = re.findall(r"(?:[A-Z][^.]*\bwill\b[^.]*\.)", text)
        return "\n".join(actions) if actions else "No clear action items detected."
    except Exception as e:
        return f"[Error extracting action items]: {str(e)}"

def process_meeting_audio(audio_path):
    transcript = transcribe_audio(audio_path)
    summary = summarize_text(transcript)
    actions = extract_action_items(transcript)
    return transcript, summary, actions


def safe_load_image(path, fallback_text):
    try:
        return Image.open(path)
    except:
        print(f"⚠️ Couldn't load image: {path}")
        return None

logo = safe_load_image("/content/logo.png", "MeetMate")
img_transcript = safe_load_image("/content/transcription.webp", "Transcript")
img_summary = safe_load_image("/content/summary.png", "Summary")
img_actions = safe_load_image("/content/action.jpg", "Action Items")


def handle_audio(audio_path):
    transcript, summary, actions = process_meeting_audio(audio_path)
    return img_transcript, transcript, img_summary, summary, img_actions, actions

with gr.Blocks() as demo:
    if logo:
        gr.Image(value=logo, show_label=False)
    gr.Markdown("## 🎤 Upload Your Meeting Audio (.mp3/.wav)")

    # Removed the 'source' argument as it's not accepted by the constructor
    audio_input = gr.Audio(type="filepath", label="Meeting Audio")
    process_button = gr.Button("🚀 Process Meeting")

    with gr.Row():
        out_img_1 = gr.Image()
        out_transcript = gr.Textbox(label="📝 Transcript", lines=6)

    with gr.Row():
        out_img_2 = gr.Image()
        out_summary = gr.Textbox(label="📄 Summary", lines=4)

    with gr.Row():
        out_img_3 = gr.Image()
        out_actions = gr.Textbox(label="✅ Action Items", lines=4)

    process_button.click(fn=handle_audio, inputs=audio_input,
                         outputs=[
                             out_img_1, out_transcript,
                             out_img_2, out_summary,
                             out_img_3, out_actions
                         ])

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1872dc79ca11f2e7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
